In [1]:
import sys
from pathlib import Path
from datetime import datetime

# Añade src al path
sys.path.append(str(Path().resolve().parent / "src"))


In [2]:
# para actualizar cambios del .py en el IPython notebook
%reload_ext autoreload
%autoreload 2

In [3]:
from data import load_last_12_months_data

rides_all = load_last_12_months_data(datetime(2024, 2, 1))
rides_all['pickup_datetime'].describe()

Descargando datos de 2023-02
El archivo ya existe: /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/data/raw/rides_2023_02.parquet
Datos cargados desde ../data/raw/rides_2023_02.parquet
Descargando datos de 2023-03
El archivo ya existe: /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/data/raw/rides_2023_03.parquet
Datos cargados desde ../data/raw/rides_2023_03.parquet
Descargando datos de 2023-04
El archivo ya existe: /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/data/raw/rides_2023_04.parquet
Datos cargados desde ../data/raw/rides_2023_04.parquet
Descargando datos de 2023-05
El archivo ya existe: /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/data/raw/rides_2023_05.parquet
Datos cargados desde ../data/raw/rides_2023_05.parquet
Descargando datos de 2023-06
El archivo ya existe: /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/data/raw/rides_2023_06.parquet
Datos cargados desde ../data/raw/rides_2

count                      38207384
mean     2023-07-31 17:30:57.726708
min             2023-02-01 00:00:00
25%      2023-04-30 12:54:37.750000
50%             2023-07-29 13:57:25
75%             2023-11-02 01:19:33
max             2024-01-31 23:59:55
Name: pickup_datetime, dtype: object

In [4]:
rides_all['pickup_datetime'].describe(include='all')

count                      38207384
mean     2023-07-31 17:30:57.726708
min             2023-02-01 00:00:00
25%      2023-04-30 12:54:37.750000
50%             2023-07-29 13:57:25
75%             2023-11-02 01:19:33
max             2024-01-31 23:59:55
Name: pickup_datetime, dtype: object

In [5]:
from data import transform_to_time_series

# Transformamos el dataframe a una serie temporal
ts_data = transform_to_time_series(rides_all)
ts_data

/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:148: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:113: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(
Rellenando huecos por localización: 100%|██████████| 263/263 [00:00<00:00, 445.82it/s]


,pickup_hour,pickup_location_id,rides
0,2023-02-01 00:00:00,4,4
1,2023-02-01 01:00:00,4,7
2,2023-02-01 02:00:00,4,0
3,2023-02-01 03:00:00,4,0
4,2023-02-01 04:00:00,4,0
...,...,...,...
2303875,2024-01-31 19:00:00,110,0
2303876,2024-01-31 20:00:00,110,0
2303877,2024-01-31 21:00:00,110,0
2303878,2024-01-31 22:00:00,110,0


In [6]:
ts_data[ts_data.pickup_location_id == 138]

,pickup_hour,pickup_location_id,rides
324120,2023-02-01 00:00:00,138,33
324121,2023-02-01 01:00:00,138,6
324122,2023-02-01 02:00:00,138,8
324123,2023-02-01 03:00:00,138,4
324124,2023-02-01 04:00:00,138,0
...,...,...,...
332875,2024-01-31 19:00:00,138,200
332876,2024-01-31 20:00:00,138,205
332877,2024-01-31 21:00:00,138,137
332878,2024-01-31 22:00:00,138,222


In [7]:
from data import transform_to_features_and_target

# Transformamos la serie temporal a un dataframe de características y objetivo
location_id = 138
n_lags = 24*28 # 1 días

X, y, df_location = transform_to_features_and_target(ts_data, location_id, n_lags)
df_location


/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'rides_previous_{lag}_hour'] = df['rides'].shift(lag)
/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'rides_previous_{lag}_hour'] = df['rides'].shift(lag)
/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

,pickup_hour,pickup_location_id,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,...,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,target
0,2023-03-01 00:00:00,138,33.0,6.0,8.0,4.0,0.0,2.0,0.0,33.0,...,248.0,252.0,161.0,193.0,215.0,223.0,218.0,253.0,282.0,106
1,2023-03-01 01:00:00,138,6.0,8.0,4.0,0.0,2.0,0.0,33.0,46.0,...,252.0,161.0,193.0,215.0,223.0,218.0,253.0,282.0,106.0,6
2,2023-03-01 02:00:00,138,8.0,4.0,0.0,2.0,0.0,33.0,46.0,115.0,...,161.0,193.0,215.0,223.0,218.0,253.0,282.0,106.0,6.0,0
3,2023-03-01 03:00:00,138,4.0,0.0,2.0,0.0,33.0,46.0,115.0,221.0,...,193.0,215.0,223.0,218.0,253.0,282.0,106.0,6.0,0.0,1
4,2023-03-01 04:00:00,138,0.0,2.0,0.0,33.0,46.0,115.0,221.0,102.0,...,215.0,223.0,218.0,253.0,282.0,106.0,6.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8083,2024-01-31 19:00:00,138,238.0,151.0,95.0,116.0,150.0,80.0,12.0,22.0,...,152.0,132.0,157.0,143.0,200.0,170.0,145.0,142.0,197.0,200
8084,2024-01-31 20:00:00,138,151.0,95.0,116.0,150.0,80.0,12.0,22.0,4.0,...,132.0,157.0,143.0,200.0,170.0,145.0,142.0,197.0,200.0,205
8085,2024-01-31 21:00:00,138,95.0,116.0,150.0,80.0,12.0,22.0,4.0,0.0,...,157.0,143.0,200.0,170.0,145.0,142.0,197.0,200.0,205.0,137
8086,2024-01-31 22:00:00,138,116.0,150.0,80.0,12.0,22.0,4.0,0.0,4.0,...,143.0,200.0,170.0,145.0,142.0,197.0,200.0,205.0,137.0,222


In [8]:
# guardamos datos
from paths import TRANSFORMED_DATA_DIR

df_location.to_parquet(TRANSFORMED_DATA_DIR / 'tabular_data_1year_4weeks_lags.parquet')